# Example to illustrate how to use WQChartPy

## Introduction

This Jupyter file was created to illustrated how to use WQChartPy by using the example geochemical dataset taken from Yang et al. (2020). This dataset contains a total of 1,184 groundwater samples. Yang et al. (2020) used this dataset to to reveal spatio-temporal patterns and controlling factors of groundwater geochemistry in Jianghan Plain, China.

## Load data

In [1]:
# Import modules
import pandas as pd
import os
from scipy import stats

import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from scipy.cluster import hierarchy

from sklearn.cluster import AgglomerativeClustering
from funciones import creardf_sc


# Load the template data
raw_df = pd.read_csv('Muestras_Todas2.csv',',')

C:\Users\cristobal.machuca\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
# Show the number of water samples
print('Totla number of samples: %d' %len(raw_df))

Totla number of samples: 323


In [3]:
# Show the first five rows
raw_df.head(69)

,Contexto_H,SubCuenca,Cod_Muestr,Cod_SNGM,Nombre_Pto,uTM_Este,uTM_Norte,Elv,Tipo_Pto,Categoria,...,CeY,Anion,Cation,K_NaCl,Ca_MgSO4HC,RAS_1,Infoqca,Codlab,InfoOD,Na100
0,B1,Rio Grande Medio,LIM-CE-001,SNGM-LMR-0139,Pozo 3 AdV Montepatria,312504,6602125,410.000000,Subterranea,Pozo,...,1.163365,Bicarbonatada,calcica,0.879339,0.547381,1.093923,2019-050,8389-19,IE 047-19,21.874191
1,B1,Rio Guatulame,LIM-CE-002,SNGM-LMR-0140,Pozo 4 AdV El Palqui,313307,6594078,402.000000,Subterranea,Pozo,...,1.111604,Bicarbonatada,calcica,0.649787,0.637618,1.006428,2019-050,8390-19,IE 047-19,22.700581
2,B1,Rio Grande Alto,LIM-CE-003,SNGM-LMR-0022,Rio Grande despues de rio Carrizal,348434,6567281,1387.000000,Superficial,Rio,...,1.073967,Bicarbonatada,calcica,0.034908,-0.057888,0.130447,2019-52,8437-19,IE 044-19,8.793772
3,B1,Rio Grande Alto,LIM-CE-004,SNGM-LMR-0023,Rio Grande en Las Ramadas,348738,6567586,1380.000000,Superficial,Rio,...,1.207177,Mixta,calcica,0.101119,-0.219798,0.700478,2019-52,8438-19,IE 044-19,27.180857
4,B1,Rio Grande Alto,LIM-CE-005,SNGM-LMR-0025,Rio Carrizal antes de rio Grande,348879,6567337,1381.000000,Superficial,Rio,...,0.993652,Bicarbonatada,calcica,0.293887,-0.284034,0.387401,2019-52,8439-19,IE 044-19,18.564546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,B1,Rio Hurtado,LIM-FF-040,SNGM-LMR-0020,Rio Hurtado en Las Breas,345261,6638469,1609.000000,Superficial,Rio,...,1.374502,Sulfatada,calcica,0.188707,-0.300286,0.287461,2019-031,8333-19,IE 039-19,10.572018
65,B1,Rio Hurtado,LIM-FF-041,SNGM-LMR-0149,Pozo 2 Nuevo APR Hurtado - El Chanar,343208,6647164,1343.000000,Subterranea,Pozo,...,1.237395,Bicarbonatada,calcica,0.524087,-0.723229,0.510380,2019-031,8334-19,IE 039-19,15.673408
66,C1,Rio Hurtado,LIM-FF-042,SNGM-LMR-0052,Embalse Recoleta en caleta sur,299783,6624739,389.156708,Superficial,Embalse,...,1.232630,Mixta,calcica,0.464625,-0.806667,0.625621,2019-031,8335-19,IE 039-19,17.540383
67,A1,Rio Hurtado,LIM-FF-043,SNGM-LMR-0130,Pozo 1 ESSCO en Huamalata,293012,6616058,250.525299,Subterranea,Pozo,...,1.192913,Mixta,calcica,0.125236,-0.095571,0.883115,2019-031,8336-19,IE 039-19,22.365260


One may note that this dataset is the raw data. We will perform the Clustering analysis by following Yang et al. (2020) first and show below how to process the raw data and the use WQChartPy to display the samples.

## Clustering the data using HCA

### Data preprocessing

In [4]:
# Get sample ID
Y_df = pd.read_csv('Muestras_Todas2.csv',',', header=0)[['Cod_Muestr','Cu_mg_l', 'Cr_mg_l', 'F_mg_l', 'Fe_mg_l', 'Mn_mg_l', 'Mg_mg_l', 'Se_mg_l', 'Zn_mg_l', 'As_mg_l', 'Cd_mg_l','Hg_mg_l', 'NO3_mg_l','Pb_mg_l','Cl_mg_l','SO4_mg_l','STD_mg_l']]

C:\Users\cristobal.machuca\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


### Clustering the geochemical data by using HCA

In [5]:
Y_df

,Cod_Muestr,Cu_mg_l,Cr_mg_l,F_mg_l,Fe_mg_l,Mn_mg_l,Mg_mg_l,Se_mg_l,Zn_mg_l,As_mg_l,Cd_mg_l,Hg_mg_l,NO3_mg_l,Pb_mg_l,Cl_mg_l,SO4_mg_l,STD_mg_l
0,LIM-CE-001,0.000,0.0000,0.16,0.000,0.0000,22.5,0.0000,0.000,0.0000,0.000,0.00015,29.320,0.000,45.884,136.375,570.0
1,LIM-CE-002,0.000,0.0000,0.15,0.000,0.0000,20.5,0.0000,0.000,0.0000,0.000,0.00015,40.292,0.000,38.647,80.478,445.0
2,LIM-CE-003,0.005,0.0005,0.10,0.005,0.0015,0.5,0.0025,0.005,0.0070,0.001,0.00015,1.500,0.001,2.300,8.600,57.0
3,LIM-CE-004,0.005,0.0005,0.12,0.005,0.0015,3.5,0.0025,0.005,0.0090,0.001,0.00015,1.500,0.001,20.000,42.000,150.0
4,LIM-CE-005,0.005,0.0005,0.04,0.005,0.0015,4.5,0.0025,0.005,0.0025,0.001,0.00015,1.500,0.001,1.600,18.000,103.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,LIM-NC-012,0.000,0.0000,0.07,0.000,0.0000,10.0,0.0000,0.000,0.0000,0.000,0.00015,1.500,0.000,2.979,89.871,243.0
319,LIM-NC-013,0.000,0.0000,0.10,0.000,0.0000,7.0,0.0000,0.000,0.0000,0.000,0.00015,1.500,0.000,3.164,110.437,263.0
320,LIM-NC-014,0.000,0.0000,0.23,0.000,0.0000,100.0,0.0000,0.000,0.0000,0.000,0.00396,23.852,0.000,139.361,841.901,1680.0
321,LIM-NC-015,0.000,0.0000,0.09,0.000,0.0000,7.5,0.0000,0.000,0.0000,0.000,0.00015,1.500,0.000,5.231,19.278,122.0


In [6]:
tilde=['Río Hurtado','Río Limarí', 'Río Grande Alto','Río Grande Medio', 'Río Grande Bajo','Río Guatulame','F.A.E']
nut=['Rio Hurtado','Rio Limari', 'Rio Grande Alto','Rio Grande Medio', 'Rio Grande Bajo','Rio Guatulame','Fuera']

dict_tilde=dict(zip(nut,tilde))
dict2= dict({'Criosfera': 'criósfera', 'Precipitacion': 'precipitación', 'Proceso': 'Proceso', 'Subterranea': 'subterránea', 'Superficial': 'superficial', 'Vertiente': 'vertiente'})
y_m= raw_df['SubCuenca'].copy()
y_f= raw_df['Contexto_H'].copy()
y_t= raw_df['Tipo_Pto'].copy()
y_tds= raw_df['STD_mg_l'].copy()
ind=0
for i in raw_df['SubCuenca']:
    y_t.at[ind]= dict2[raw_df['Tipo_Pto'][ind]]
    y_m.at[ind]= dict_tilde[i]
    if y_f.at[ind]== 'Fuera del area de estudio':
        y_f.at[ind]='F.A.E'
    ind+=1
#print (y_m)

In [7]:
# Group the data into seven clusters
pd.set_option('display.max_rows', 20)
cluster = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward')  
y_seven = y_m.copy()
y_2= y_f.copy()
y_seven

0      Río Grande Medio
1         Río Guatulame
2       Río Grande Alto
3       Río Grande Alto
4       Río Grande Alto
             ...       
318    Río Grande Medio
319    Río Grande Medio
320          Río Limarí
321       Río Guatulame
322     Río Grande Bajo
Name: SubCuenca, Length: 323, dtype: object

In [12]:
# Save the clustering results
Y_df['SubCuenca'] = y_seven
Y_df['Contexto_H'] = y_2
Y_df['Tipo_Pto']= y_t
Y_df['TDS']= y_tds
# Show Y_df
#Y_df = Y_df.replace("NaN", 0.0005)
#Y_df = Y_df.dropna(axis=0, how='any')
Y_df

,Cod_Muestr,Cu_mg_l,Cr_mg_l,F_mg_l,Fe_mg_l,Mn_mg_l,Mg_mg_l,Se_mg_l,Zn_mg_l,As_mg_l,...,Hg_mg_l,NO3_mg_l,Pb_mg_l,Cl_mg_l,SO4_mg_l,STD_mg_l,SubCuenca,Contexto_H,Tipo_Pto,TDS
0,LIM-CE-001,0.000,0.0000,0.16,0.000,0.0000,22.5,0.0000,0.000,0.0000,...,0.00015,29.320,0.000,45.884,136.375,570.0,Río Grande Medio,B1,subterránea,570.0
1,LIM-CE-002,0.000,0.0000,0.15,0.000,0.0000,20.5,0.0000,0.000,0.0000,...,0.00015,40.292,0.000,38.647,80.478,445.0,Río Guatulame,B1,subterránea,445.0
2,LIM-CE-003,0.005,0.0005,0.10,0.005,0.0015,0.5,0.0025,0.005,0.0070,...,0.00015,1.500,0.001,2.300,8.600,57.0,Río Grande Alto,B1,superficial,57.0
3,LIM-CE-004,0.005,0.0005,0.12,0.005,0.0015,3.5,0.0025,0.005,0.0090,...,0.00015,1.500,0.001,20.000,42.000,150.0,Río Grande Alto,B1,superficial,150.0
4,LIM-CE-005,0.005,0.0005,0.04,0.005,0.0015,4.5,0.0025,0.005,0.0025,...,0.00015,1.500,0.001,1.600,18.000,103.0,Río Grande Alto,B1,superficial,103.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,LIM-NC-012,0.000,0.0000,0.07,0.000,0.0000,10.0,0.0000,0.000,0.0000,...,0.00015,1.500,0.000,2.979,89.871,243.0,Río Grande Medio,B1,subterránea,243.0
319,LIM-NC-013,0.000,0.0000,0.10,0.000,0.0000,7.0,0.0000,0.000,0.0000,...,0.00015,1.500,0.000,3.164,110.437,263.0,Río Grande Medio,B1,superficial,263.0
320,LIM-NC-014,0.000,0.0000,0.23,0.000,0.0000,100.0,0.0000,0.000,0.0000,...,0.00396,23.852,0.000,139.361,841.901,1680.0,Río Limarí,A1,subterránea,1680.0
321,LIM-NC-015,0.000,0.0000,0.09,0.000,0.0000,7.5,0.0000,0.000,0.0000,...,0.00015,1.500,0.000,5.231,19.278,122.0,Río Guatulame,B2,superficial,122.0


### Organize the data into WQChartPy required formats

## Display the samples by using WQChartPy

In [9]:
# Import WQChartPy
#from wqchartpy import schoellermod

#format_df,filtro,filtro2 = creardf_sc(Y_df,'F.A.E','',8)
# Draw the diagram
#schoellermod.plot(format_df, unit='mg/L', figname='Schoeller '+filtro+'_'+filtro2, figformat='png',ms=np.nan,n=True,nch='Nch 409')
#plt.savefig('Schoeller2 '+filtro+'_'+filtro2 + '.' + 'jpg', format='jpg', bbox_inches='tight', dpi=300)

In [14]:
pd.set_option("display.max_rows", None
              , "display.max_columns",None)
data2 = {'Sample' : ['Nch409', 'Nch1333'],
            'Label'  : ['Nch409', 'Nch1333'],
            'Size'   : [np.nan, np.nan],
            'Alpha'  : [1, 1],
            'Cu'     : [2.0, 0.2],
            'Cr'     : [0.05, 0.1],
            'F'     : [1.5, 1.0],
            'Fe'     : [0.3, 5.0],
            'Mn'      : [0.1, 0.2],
            'Mg'   : [125.0, 9999],
            'Se'    : [0.01, 0.02],
            'Zn'    : [3.0, 2.0],
            'As'    : [0.01, 0.1],
            'Cd'    : [0.01, 0.01],
            'Hg'    : [0.001, 0.001],
            'NO3'    : [50, 9999],
            'Pb'    : [0.05, 5],
            'Cl'    : [400, 200],
            'SO4'    : [500, 250],
            'TDS'    : [1500, 5000],
            }
def sobre_norma(data, formato):
    
    df = pd.DataFrame(data)
    
    lista1=list()
    listaconteo=list()
    for a in range(0,len(df),1):
        df2 = pd.DataFrame()   
        for i in range(0,len(formato),1):
            df2=df2.append([df.iloc[a]])
            
        df2.reset_index(inplace=True, drop=True)

        new=pd.DataFrame()
        new['Sample']=formato['Sample']
#print (len(format_df['Cu']))
       
#new['Cu']= format_df['Cu']>=df2['Cu']

        new['valor']=pd.DataFrame({'valor':[False]*len(format_df)})

        for i in df2.columns:
            if df2[i].dtypes != 'object':
                new[i]= formato[i]>=df2[i]
                new['valor']=(new['valor'] | new[i])


        df2=new.groupby('valor').get_group(True)
        df3=pd.DataFrame(df2.any(axis=0)).T
        df4=pd.DataFrame()
        df4['valor2']=df2['valor'].reset_index(drop=True)
        for i in df3.columns:
            tdf=(df2[df2[i]==True]['Sample']).reset_index(drop=True)
            tdf2=(pd.DataFrame(tdf)).rename(columns={'Sample':i})

            df4=df4.join(tdf2)
        eli=['valor2','Sample','valor','Size','Alpha']
        for i in eli:
            df4.pop(i)
        df4.dropna(how='all',inplace=True)
        df4.dropna(axis=1,how='all',inplace=True)
        conteo=pd.DataFrame()
        #print (df4)
        for i in df4.columns:
            #print (i)
            conteo[i]=[df4[i].count()]
            #print (conteo[i])
        lista1.append(df4)
        listaconteo.append(conteo)
        #print ('a')
    conteofin=listaconteo[0].append(listaconteo[1])
    conteofin.reset_index(drop=True,inplace=True)
    dfg=conteofin.rename(index={0:'Nch 409',1:'Nch 1333'})
    return lista1 , dfg



In [25]:
from wqchartpy import schoellermod
sbc=['Río Hurtado']#,'Río Grande Medio','Río Grande Alto','Río Grande Bajo','Río Guatulame','Río Limarí','F.A.E']
tpt=['criósfera']#['superficial','subterránea','vertiente']
ky=Y_df.groupby(['SubCuenca','Tipo_Pto']).groups.keys()
ky2=Y_df.groupby(['SubCuenca']).groups.keys()
i=0
for sc in sbc:
    for t in tpt:
        if (sc,t) in ky:
            #print ('Diagrama Schoeller Subcuenca '+filtro+'/ '+filtro2)
            format_df,filtro,filtro2 = creardf_sc(Y_df,sc,t,8)
            #schoellermod.plot(format_df, unit='mg/L', figname='Subcuenca '+filtro+'-'+filtro2, figformat='png',ms=np.nan,n=True,nch='Nch 409')
            i+=1
            j,k= sobre_norma(data2,format_df)
            k.to_csv('Scholler_{}.csv'.format('CSN_'+sc+'_'+t),sep=',')
            j[0].to_csv('Scholler_{}.csv'.format('NCH409_'+sc+'_'+t),sep=',')
            j[1].to_csv('Scholler_{}.csv'.format('NCH1333_'+sc+'_'+t),sep=',')
        elif (sc in ky2) and t=='':
            #print ('Diagrama Schoeller Subcuenca '+filtro+'/ '+filtro2)
            format_df,filtro,filtro2 = creardf_sc(Y_df,sc,t,8)
            #schoellermod.plot(format_df, unit='mg/L', figname='Subcuenca '+filtro+'-'+filtro2, figformat='png',ms=np.nan,n=True,nch='Nch 409')
            i+=1
            j,k= sobre_norma(data2,format_df)
            #k.to_csv('Scholler_{}.csv'.format('CSN_'+sc+'_'+t),sep=',')
            j[0].to_csv('Scholler_{}.csv'.format('NCH409_'+sc+'_'+t),sep=',')
            j[1].to_csv('Scholler_{}.csv'.format('NCH1333_'+sc+'_'+t),sep=',')
            
        else: continue
            
        plt.savefig(str(i)+'_Schoeller_'+sc+'_'+t+'.' + 'jpg', format='jpg', bbox_inches='tight', dpi=300)


<Figure size 640x480 with 0 Axes>

In [ ]:
j,k= sobre_norma(data2,format_df)
k.to_csv('Scholler_{}.csv'.format('cantidad_sobre_norma'),sep=',')
j[0].to_csv('Scholler_{}.csv'.format('M_NCH409'),sep=',')
j[1].to_csv('Scholler_{}.csv'.format('M_NCH1333'),sep=',')

In [13]:
tp=raw_df.groupby(['Tipo_Pto'])
lista=['Precipitacion','Proceso']
dp=raw_df.copy()
for i in lista:
    p=tp.get_group(i)
    sp=dp.drop(p.index)
    dp=sp
gb=sp.groupby(['Contexto_H'])
l=list(gb.groups.keys())

In [28]:
sbc=['Rio Hurtado','Rio Grande Medio','Rio Grande Alto','Rio Grande Bajo','Rio Guatulame','Rio Limari','Fuera']
tpt=['']#['superficial','subterránea','vertiente']
sp['TDS']=sp['STD_mg_l'].copy()
sp['d']=sp['Deuterio']-(sp['Oxigeno_18']*8)
DF=sp.groupby(['SubCuenca','Tipo_Pto'])
DF2= sp.groupby(['SubCuenca'])
ky=DF.groups.keys()
ky2=DF2.groups.keys()
#print (ky2)
dfs=list()
i=0
for sc in sbc:
    cta=list()
    for t in tpt:
        if (sc,t) in ky:
            #print ('Diagrama Schoeller Subcuenca '+filtro+'/ '+filtro2)
            format_df,filtro,filtro2 = creardf_sc(sp,sc,t,8)
            i+=1
            
            format_df.to_csv('Scatter_{}.csv'.format(sc+'_'+t),sep=',')
            format_df.to_csv('Scatter_{}.csv'.format(sc+'_'+t),sep=',')
        elif (sc in ky2) and t=='':
            
            #format_df,filtro,filtro2 = creardf_sc(sp,sc,t,8)
            i+=1
            
            #DF.get_group(sc).to_csv('Scatter_{}.csv'.format(sc+'_'+t),sep=',')
            #DF2.get_group(sc).to_csv('Scartter_{}.csv'.format(sc+'_'+t),sep=',')
            print (sc+' '+t)
            
            #orden por cantidad de clases
            sg=DF2.get_group(sc)['Tipo_Piper'].value_counts()
            sg.name=sc
            dfs.append(sg)
            
            #valores de isotopos
            varis=['Oxigeno_18','Deuterio','Ex_deuteri']
            var=varis[2]    
            #print (str(max(DF2.get_group(sc)['d']))[:5]+'/'+str(min(DF2.get_group(sc)['d']))[:5])
            print (str(max(DF2.get_group(sc)[var]))+'/'+str(min(DF2.get_group(sc)[var])))
            DF2.get_group(sc).columns
        else: 
            #print ('aaaa')
            continue
#raw_df.to_csv('Scatter_{}.csv'.format('final'),sep=',')            
        #plt.savefig(str(i)+'_Schoeller_'+sc+'_'+t+'.' + 'jpg', format='jpg', bbox_inches='tight', dpi=300)
#a=list(DF2.get_group(sc).columns)
a= pd.DataFrame(dfs).T
a['Limari/Grande Bajo']=a['Rio Limari']+a['Rio Grande Bajo'].fillna(0)
for i in a.columns:
    b=a[i].sort_values(ascending=False)
    c=b.dropna()
    print (i)
    print (list(c.index.values))
a

Rio Hurtado 
15.33/-69.0
Rio Grande Medio 
13.42/-6.01
Rio Grande Alto 
12.62/5.87
Rio Grande Bajo 
5.61/-3.7
Rio Guatulame 
9.24/-6.79
Rio Limari 
14.03/-3.23
Fuera 
14.7/2.56
Rio Hurtado
['Sulfatada calcica', 'Mixta calcica', 'Bicarbonatada calcica', 'Bicarbonatada mixta', 'Clorurada magnesica', 'Mixta']
Rio Grande Medio
['Bicarbonatada calcica', 'Sulfatada calcica', 'Bicarbonatada mixta', 'Mixta calcica', 'Mixta']
Rio Grande Alto
['Bicarbonatada calcica', 'Sulfatada calcica', 'Mixta calcica']
Rio Grande Bajo
['Bicarbonatada calcica', 'Bicarbonatada mixta']
Rio Guatulame
['Bicarbonatada calcica', 'Bicarbonatada mixta', 'Sulfatada sodica', 'Mixta', '- -', 'Sulfatada magnesica']
Rio Limari
['Clorurada sodica', 'Mixta', 'Clorurada mixta', 'Bicarbonatada mixta', 'Mixta sodica', 'Sulfatada mixta', 'Sulfatada calcica', 'Bicarbonatada magnesica', 'Bicarbonatada calcica', 'Sulfatada sodica']
Fuera
['Bicarbonatada mixta', 'Mixta', 'Clorurada sodica', 'Clorurada mixta']
Limari/Grande Bajo
['Cl

,Rio Hurtado,Rio Grande Medio,Rio Grande Alto,Rio Grande Bajo,Rio Guatulame,Rio Limari,Fuera,Limari/Grande Bajo
Sulfatada calcica,25.0,8.0,2.0,NaN,NaN,2.0,NaN,2.0
Mixta calcica,15.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN
Bicarbonatada calcica,9.0,15.0,8.0,4.0,32.0,1.0,NaN,5.0
Bicarbonatada mixta,2.0,4.0,NaN,2.0,6.0,20.0,9.0,22.0
Clorurada magnesica,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mixta,1.0,1.0,NaN,NaN,1.0,26.0,4.0,26.0
Sulfatada sodica,NaN,NaN,NaN,NaN,2.0,1.0,NaN,1.0
- -,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
Sulfatada magnesica,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
Clorurada sodica,NaN,NaN,NaN,NaN,NaN,33.0,3.0,33.0


In [81]:

sbc=['Rio Hurtado','Rio Grande Medio','Rio Grande Alto','Rio Grande Bajo','Rio Guatulame','Rio Limari','Fuera']
tpt=l#['superficial','subterránea','vertiente']
sp['TDS']=sp['STD_mg_l'].copy()
sp['d']=sp['Deuterio']-(sp['Oxigeno_18']*8)
DF=sp.groupby(['SubCuenca','Contexto_H'])
DF2= sp.groupby(['SubCuenca'])
ky=DF.groups.keys()
ky2=DF2.groups.keys()
#print (ky2)
dfs=list()

df=pd.DataFrame()
i=0
varis=['Oxigeno_18','Deuterio','Ex_deuteri']
for var in varis:
    dfsmax=list()
    dfsmin=list()
    cta=list()
    for sc in sbc:
        
        for t in tpt:
            tupla=(sc,t)
            if tupla in ky:
                #sg=DF.get_group((sc,t))['Tipo_Piper'].value_counts()
                #sg.name=sc
                #dfs.append(sg)
                #print (sc+' '+t)
                #valores de isotopos
            
                dfsmax.append(max(DF.get_group(tupla)[var]))
                dfsmin.append(min(DF.get_group(tupla)[var]))
                #print (str(max(DF2.get_group(sc)['d']))[:5]+'/'+str(min(DF2.get_group(sc)['d']))[:5])
                #print ('     '+var+' '+str(max(DF.get_group(tupla)[var]))+'/'+str(min(DF.get_group(tupla)[var])))
                #DF.get_group(tupla).columns
                #cta.append(tupla[0]+'/'+tupla[1])
                cta.append(tupla)
                
            elif (sc in ky2) and t=='':
            
                format_df,filtro,filtro2 = creardf_sc(raw_df,sc,t,8)
                i+=1
            
                #DF.get_group(sc).to_csv('Scatter_{}.csv'.format(sc+'_'+t),sep=',')
                #DF2.get_group(sc).to_csv('Scartter_{}.csv'.format(sc+'_'+t),sep=',')
                print (sc+' '+t)
            
                #orden por cantidad de clases
                sg=DF2.get_group(sc)['Tipo_Piper'].value_counts()
                sg.name=sc
                dfs.append(sg)
            
                #valores de isotopos
                varis=['Oxigeno_18','Deuterio','Ex_deuteri']
                var=varis[0]    
                #print (str(max(DF2.get_group(sc)['d']))[:5]+'/'+str(min(DF2.get_group(sc)['d']))[:5])
                print (str(max(DF2.get_group(sc)[var]))+'/'+str(min(DF2.get_group(sc)[var])))
                DF2.get_group(sc).columns
            else: continue
    pdmaxi_index=pd.MultiIndex.from_tuples(cta, names=["Subcuenca", "UH"])          
    pdmaxi=pd.Series(dfsmax,name=var+' max', index=pdmaxi_index)
    #pdmaxi.index=cta
     
    pdmin=pd.Series(dfsmin,name=var+' min', index=pdmaxi_index)
    #pdmin.index=cta
    df=pd.concat([df,pdmaxi,pdmin],axis=1)
    df.index=pdmaxi_index
            #raw_df.to_csv('Scatter_{}.csv'.format('final'),sep=',')            
        #plt.savefig(str(i)+'_Schoeller_'+sc+'_'+t+'.' + 'jpg', format='jpg', bbox_inches='tight', dpi=300)
#a=list(DF2.get_group(sc).columns)
#a= pd.DataFrame(dfs).T
#a['Limari/Grande Bajo']=a['Rio Limari']+a['Rio Grande Bajo'].fillna(0)
#for i in a.columns:
#    b=a[i].sort_values(ascending=False)
#    c=b.dropna()
#    print (i)
#    print (list(c.index.values))
df.to_csv('Lista_{}.csv'.format('max_min'),sep=',')
df

Oxigeno_18 max  Oxigeno_18 min  \
Subcuenca        UH                                                          
Rio Hurtado      A1                                  -7.46           -9.23   
                 B1                                  11.51          -17.23   
                 C1                                  -8.13           -8.34   
Rio Grande Medio B1                                  -6.30          -14.76   
                 B2                                  -6.13           -7.06   
                 D1                                  -9.20           -9.24   
                 D3                                 -10.18          -10.58   
Rio Grande Alto  B1                                 -11.56          -13.08   
Rio Grande Bajo  A1                                  -7.78           -9.20   
Rio Guatulame    A3                                  -7.11          -10.30   
                 B1                                  -6.40          -11.57   
                 B2                                  -5.54           -5.59   
                 D3                                  -8.70           -8.70   
Rio Limari       A1                                  -0.24           -8.76   
                 A2                                  -4.52           -8.54   
                 A3                                  -4.65           -6.90   
                 B1                                  -6.42           -6.96   
                 B2                                  -6.79           -7.51   
                 B4                                  -4.04           -5.92   
                 D1                                  -0.50           -0.50   
Fuera            Fuera del area de estudio           -4.34           -5.72   

                                            Deuterio max  Deuterio min  \
Subcuenca        UH                                                      
Rio Hurtado      A1                                -55.4         -76.2   
                 B1                                 23.1        -127.5   
                 C1                                -66.8         -70.4   
Rio Grande Medio B1                                -56.4        -110.4   
                 B2                                -49.3         -55.8   
                 D1                                -67.4         -68.4   
                 D3                                -75.1         -77.5   
Rio Grande Alto  B1                                -82.2         -98.3   
Rio Grande Bajo  A1                                -65.9         -72.2   
Rio Guatulame    A3                                -48.6         -75.0   
                 B1                                -46.5         -85.8   
                 B2                                -50.8         -51.5   
                 D3                                -66.2         -66.2   
Rio Limari       A1                                 -1.2         -70.1   
                 A2                                -29.5         -65.6   
                 A3                                -27.8         -43.8   
                 B1                                -41.1         -46.6   
                 B2                                -48.0         -52.0   
                 B4                                -28.5         -38.6   
                 D1                                 -0.6          -0.6   
Fuera            Fuera del area de estudio         -28.4         -38.6   

                                            Ex_deuteri max  Ex_deuteri min  
Subcuenca        UH                                                         
Rio Hurtado      A1                                   5.74           -2.34  
                 B1                                  15.33          -69.00  
                 C1                                  -1.72           -3.67  
Rio Grande Medio B1                                  13.42           -6.01  
                 B2                                   2.90           -0.91  
       

In [30]:
pd.set_option("display.max_rows", 25
              , "display.max_columns",None)

sbc=l
tpt=['Rio Hurtado','Rio Grande Medio','Rio Grande Alto','Rio Grande Bajo','Rio Guatulame','Rio Limari','Fuera']#['superficial','subterránea','vertiente']

sp['TDS']=sp['STD_mg_l'].copy()
sp['d']=sp['Deuterio']-(sp['Oxigeno_18']*8)
DF=sp.groupby(['Contexto_H','SubCuenca',])
DF2= sp.groupby(['Contexto_H'])
ky=DF.groups.keys()
ky2=DF2.groups.keys()
#print (ky2)
dfs=list()

df=pd.DataFrame()
i=0
varis=['Oxigeno_18','Deuterio','Ex_deuteri']
for var in varis:
    dfsmax=list()
    dfsmin=list()
    cta=list()
    for sc in sbc:
        
        for t in tpt:
            tupla=(sc,t)
            if tupla in ky:
                #sg=DF.get_group((sc,t))['Tipo_Piper'].value_counts()
                #sg.name=sc
                #dfs.append(sg)
                #print (sc+' '+t)
                #valores de isotopos
            
                dfsmax.append(max(DF.get_group(tupla)[var]))
                dfsmin.append(min(DF.get_group(tupla)[var]))
                #print (str(max(DF2.get_group(sc)['d']))[:5]+'/'+str(min(DF2.get_group(sc)['d']))[:5])
                #print ('     '+var+' '+str(max(DF.get_group(tupla)[var]))+'/'+str(min(DF.get_group(tupla)[var])))
                #DF.get_group(tupla).columns
                #cta.append(tupla[0]+'/'+tupla[1])
                cta.append(tupla)
                
            elif (sc in ky2) and t=='':
            
                format_df,filtro,filtro2 = creardf_sc(raw_df,sc,t,8)
                i+=1
            
                #DF.get_group(sc).to_csv('Scatter_{}.csv'.format(sc+'_'+t),sep=',')
                #DF2.get_group(sc).to_csv('Scartter_{}.csv'.format(sc+'_'+t),sep=',')
                print (sc+' '+t)
            
                #orden por cantidad de clases
                sg=DF2.get_group(sc)['Tipo_Piper'].value_counts()
                sg.name=sc
                dfs.append(sg)
            
                #valores de isotopos
                varis=['Oxigeno_18','Deuterio','Ex_deuteri']
                var=varis[0]    
                #print (str(max(DF2.get_group(sc)['d']))[:5]+'/'+str(min(DF2.get_group(sc)['d']))[:5])
                print (str(max(DF2.get_group(sc)[var]))+'/'+str(min(DF2.get_group(sc)[var])))
                DF2.get_group(sc).columns
            else: continue
    pdmaxi_index=pd.MultiIndex.from_tuples(cta, names=["Subcuenca", "UH"])          
    pdmaxi=pd.Series(dfsmax,name=var+' max', index=pdmaxi_index)
    #pdmaxi.index=cta
     
    pdmin=pd.Series(dfsmin,name=var+' min', index=pdmaxi_index)
    #pdmin.index=cta
    df=pd.concat([df,pdmaxi,pdmin],axis=1)
    df.index=pdmaxi_index
            #raw_df.to_csv('Scatter_{}.csv'.format('final'),sep=',')            
        #plt.savefig(str(i)+'_Schoeller_'+sc+'_'+t+'.' + 'jpg', format='jpg', bbox_inches='tight', dpi=300)
#a=list(DF2.get_group(sc).columns)
#a= pd.DataFrame(dfs).T
#a['Limari/Grande Bajo']=a['Rio Limari']+a['Rio Grande Bajo'].fillna(0)
#for i in a.columns:
#    b=a[i].sort_values(ascending=False)
#    c=b.dropna()
#    print (i)
#    print (list(c.index.values))
#df.to_csv('Lista_{}.csv'.format('max_min'),sep=',')
df

Oxigeno_18 max  Oxigeno_18 min  \
Subcuenca                 UH                                                 
A1                        Rio Hurtado                -7.46           -9.23   
                          Rio Grande Bajo            -7.78           -9.20   
                          Rio Limari                 -0.24           -8.76   
A2                        Rio Limari                 -4.52           -8.54   
A3                        Rio Guatulame              -7.11          -10.30   
                          Rio Limari                 -4.65           -6.90   
B1                        Rio Hurtado                11.51          -17.23   
                          Rio Grande Medio           -6.30          -14.76   
                          Rio Grande Alto           -11.56          -13.08   
                          Rio Guatulame              -6.40          -11.57   
                          Rio Limari                 -6.42           -6.96   
B2                        Rio Grande Medio           -6.13           -7.06   
                          Rio Guatulame              -5.54           -5.59   
                          Rio Limari                 -6.79           -7.51   
B4                        Rio Limari                 -4.04           -5.92   
C1                        Rio Hurtado                -8.13           -8.34   
D1                        Rio Grande Medio           -9.20           -9.24   
                          Rio Limari                 -0.50           -0.50   
D3                        Rio Grande Medio          -10.18          -10.58   
                          Rio Guatulame              -8.70           -8.70   
Fuera del area de estudio Fuera                      -4.34           -5.72   

                                            Deuterio max  Deuterio min  \
Subcuenca                 UH                                             
A1                        Rio Hurtado              -55.4         -76.2   
                          Rio Grande Bajo          -65.9         -72.2   
                          Rio Limari                -1.2         -70.1   
A2                        Rio Limari               -29.5         -65.6   
A3                        Rio Guatulame            -48.6         -75.0   
                          Rio Limari               -27.8         -43.8   
B1                        Rio Hurtado               23.1        -127.5   
                          Rio Grande Medio         -56.4        -110.4   
                          Rio Grande Alto          -82.2         -98.3   
                          Rio Guatulame            -46.5         -85.8   
                          Rio Limari               -41.1         -46.6   
B2                        Rio Grande Medio         -49.3         -55.8   
                          Rio Guatulame            -50.8         -51.5   
                          Rio Limari               -48.0         -52.0   
B4                        Rio Limari               -28.5         -38.6   
C1                        Rio Hurtado              -66.8         -70.4   
D1                        Rio Grande Medio         -67.4         -68.4   
                          Rio Limari                -0.6          -0.6   
D3                        Rio Grande Medio         -75.1         -77.5   
                          Rio Guatulame            -66.2         -66.2   
Fuera del area de estudio Fuera                    -28.4         -38.6   

                                            Ex_deuteri max  Ex_deuteri min  
Subcuenca                 UH                                                
A1                        Rio Hurtado                 5.74           -2.34  
                          Rio Grande Bajo             5.61           -3.70  
                          Rio Limari                 11.23           -3.23  
A2                        Rio Limari                 12.90           -2.94  
A3                        Rio Guatulame               8.28            3.31  
       

In [33]:
sbc=l#['Rio Hurtado','Rio Grande Medio','Rio Grande Alto','Rio Grande Bajo','Rio Guatulame','Rio Limari','Fuera']
tpt=['']#['superficial','subterránea','vertiente']
sp['TDS']=sp['STD_mg_l'].copy()
sp['d']=sp['Deuterio']-(sp['Oxigeno_18']*8)
DF=sp.groupby(['Contexto_H','Tipo_Pto'])
DF2= sp.groupby(['Contexto_H'])
ky=DF.groups.keys()
ky2=DF2.groups.keys()
#print (ky2)
dfs=list()
i=0
for sc in sbc:
    cta=list()
    for t in tpt:
        if (sc,t) in ky:
            #print ('Diagrama Schoeller Subcuenca '+filtro+'/ '+filtro2)
            #format_df,filtro,filtro2 = creardf_sc(sp,sc,t,8)
            i+=1
            
            #format_df.to_csv('Scatter_{}.csv'.format(sc+'_'+t),sep=',')
            #format_df.to_csv('Scatter_{}.csv'.format(sc+'_'+t),sep=',')
        elif (sc in ky2) and t=='':
            
            #format_df,filtro,filtro2 = creardf_sc(sp,sc,t,8)
            i+=1
            
            #DF.get_group(sc).to_csv('Scatter_{}.csv'.format(sc+'_'+t),sep=',')
            #DF2.get_group(sc).to_csv('Scartter_{}.csv'.format(sc+'_'+t),sep=',')
            print (sc+' '+t)
            
            #orden por cantidad de clases
            sg=DF2.get_group(sc)['Tipo_Piper'].value_counts()
            sg.name=sc
            dfs.append(sg)
            
            #valores de isotopos
            #varis=['Oxigeno_18','Deuterio','Ex_deuteri']
            #var=varis[2]    
            #print (str(max(DF2.get_group(sc)['d']))[:5]+'/'+str(min(DF2.get_group(sc)['d']))[:5])
            #print (str(max(DF2.get_group(sc)[var]))+'/'+str(min(DF2.get_group(sc)[var])))
            #DF2.get_group(sc).columns
        else: 
            #print ('aaaa')
            continue
#raw_df.to_csv('Scatter_{}.csv'.format('final'),sep=',')            
        #plt.savefig(str(i)+'_Schoeller_'+sc+'_'+t+'.' + 'jpg', format='jpg', bbox_inches='tight', dpi=300)
#a=list(DF2.get_group(sc).columns)
a= pd.DataFrame(dfs).T
#a['Limari/Grande Bajo']=a['Rio Limari']+a['Rio Grande Bajo'].fillna(0)
for i in a.columns:
    b=a[i].sort_values(ascending=False)
    c=b.dropna()
    print (i)
    print (list(c.index.values))
a

A1 
A2 
A3 
B1 
B2 
B4 
C1 
D1 
D3 
Fuera del area de estudio 
A1
['Clorurada mixta', 'Mixta', 'Clorurada sodica', 'Bicarbonatada calcica', 'Bicarbonatada mixta', 'Sulfatada mixta', 'Mixta calcica', 'Sulfatada calcica']
A2
['Clorurada sodica', 'Mixta', 'Bicarbonatada mixta', 'Mixta sodica', 'Clorurada mixta']
A3
['Bicarbonatada mixta', 'Clorurada sodica', 'Bicarbonatada calcica', 'Mixta', 'Clorurada mixta']
B1
['Bicarbonatada calcica', 'Sulfatada calcica', 'Mixta calcica', 'Bicarbonatada mixta', 'Sulfatada sodica', 'Mixta', '- -', 'Clorurada magnesica', 'Sulfatada magnesica']
B2
['Bicarbonatada calcica', 'Bicarbonatada mixta', 'Sulfatada calcica', 'Mixta', 'Mixta calcica', 'Sulfatada sodica', 'Bicarbonatada magnesica']
B4
['Bicarbonatada mixta', 'Mixta', 'Bicarbonatada magnesica']
C1
['Mixta calcica']
D1
['Clorurada sodica', 'Bicarbonatada calcica']
D3
['Bicarbonatada calcica']
Fuera del area de estudio
['Bicarbonatada mixta', 'Mixta', 'Clorurada sodica', 'Clorurada mixta']


,A1,A2,A3,B1,B2,B4,C1,D1,D3,Fuera del area de estudio
Clorurada mixta,22.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0
Mixta,15.0,8.0,3.0,1.0,1.0,1.0,NaN,NaN,NaN,4.0
Clorurada sodica,11.0,13.0,7.0,NaN,NaN,NaN,NaN,2.0,NaN,3.0
Bicarbonatada calcica,5.0,NaN,4.0,51.0,3.0,NaN,NaN,2.0,4.0,NaN
Bicarbonatada mixta,4.0,4.0,9.0,9.0,2.0,6.0,NaN,NaN,NaN,9.0
Sulfatada mixta,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mixta calcica,2.0,NaN,NaN,14.0,1.0,NaN,2.0,NaN,NaN,NaN
Sulfatada calcica,1.0,NaN,NaN,34.0,2.0,NaN,NaN,NaN,NaN,NaN
Mixta sodica,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sulfatada sodica,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN
